In [ ]:
import mysql.connector as sql
from tkinter import *
from tkinter import messagebox
from tkinter.simpledialog import askstring
from tkinter.simpledialog import askinteger
from tkinter.simpledialog import askfloat

# Connect to the MySQL database
cnx = sql.connect(
    host='127.0.0.1',
    user='root',
    password='#Anusri7068',
    database='hotel_management_system',
    auth_plugin='mysql_native_password'
)
if not cnx.is_connected():
    print("Not connected")
else:
    print("Database connected")
curs = cnx.cursor()

def update_customer_details():
    customer_id = askinteger("Update Customer Details", "Enter the customer_id to update:")
    if not customer_id:
        return
    field_map = {
        1: "customer_name",
        2: "customer_gender",
        3: "customer_proof",
        4: "customer_email",
        5: "customer_phone_no",
        6: "customer_address"
    }
    choice = askinteger("Update Customer Details", "Which detail do you want to update?\n1. Name\n2. Gender\n3. Proof\n4. Email\n5. Phone number\n6. Address")
    field_name = field_map.get(choice)
    if not field_name:
        messagebox.showerror("Error", "Invalid choice!")
        return

    new_value = askstring("Update Customer Details", f"Enter new value for {field_name}:")
    if choice == 5:
        new_value = int(new_value)

    update_query = f"UPDATE customer_details SET {field_name} = %s WHERE customer_id = %s;"
    curs.execute(update_query, (new_value, customer_id))
    cnx.commit()
    messagebox.showinfo("Success", f"Customer details updated for customer_id {customer_id}.")

def update_employee_details():
    employee_id = askstring("Update Employee Details", "Enter the employee_id to update:")
    if not employee_id:
        return
    field_map = {
        1: "employee_name",
        2: "employee_gender",
        3: "employee_experience",
        4: "employee_position",
        5: "employee_email",
        6: "employee_phone_no",
        7: "employee_address"
    }
    choice = askinteger("Update Employee Details", "Which detail do you want to update?\n1. Name\n2. Gender\n3. Experience\n4. Position\n5. Email\n6. Phone number\n7. Address")
    field_name = field_map.get(choice)
    if not field_name:
        messagebox.showerror("Error", "Invalid choice!")
        return

    new_value = askstring("Update Employee Details", f"Enter new value for {field_name}:")
    if choice in [3, 6]:
        new_value = int(new_value)

    update_query = f"UPDATE employee_details SET {field_name} = %s WHERE employee_id = %s;"
    curs.execute(update_query, (new_value, employee_id))
    cnx.commit()
    messagebox.showinfo("Success", f"Employee details updated for employee_id {employee_id}.")

def delete_customer_details():
    customer_id = askinteger("Delete Customer Details", "Enter the customer_id to delete:")
    delete_query = "DELETE FROM customer_details WHERE customer_id = %s;"
    curs.execute(delete_query, (customer_id,))
    cnx.commit()
    messagebox.showinfo("Success", f"Customer record with customer_id {customer_id} has been deleted.")

def delete_employee_details():
    employee_id = askstring("Delete Employee Details", "Enter the employee_id to delete:")
    delete_query = "DELETE FROM employee_details WHERE employee_id = %s;"
    curs.execute(delete_query, (employee_id,))
    cnx.commit()
    messagebox.showinfo("Success", f"Employee record with employee_id {employee_id} has been deleted.")

def check_status():
    room_no = askinteger("Check Room Status", "Enter the room_no to check its status:")
    q = 'SELECT room_no, room_type, status FROM Rooms WHERE room_no = %s;'
    curs.execute(q, (room_no,))
    result = curs.fetchone()
    if result:
        room_id, room_type, status = result
        messagebox.showinfo("Room Status", f"Room {room_id} ({room_type}) is {status}.")
    else:
        messagebox.showerror("Error", "Room not found.")

def book_room():
    customer_id = askinteger("Book Room", "Enter the customer_id:")
    if not customer_id:
        return
    room_no = askinteger("Book Room", "Enter the room number to book:")
    if not room_no:
        return
    check_in_date = askstring("Book Room", "Enter the check-in date (YYYY-MM-DD):")
    check_out_date = askstring("Book Room", "Enter the check-out date (YYYY-MM-DD):")
    meal_preference = askstring("Book Room", "Do you want meals included? (yes/no):")
    if meal_preference.lower() not in ["yes", "no"]:
        messagebox.showerror("Error", "Invalid meal preference! Please enter 'yes' or 'no'.")
        return

    # Check if the room is available
    curs.execute('SELECT status FROM Rooms WHERE room_no = %s;', (room_no,))
    room_status = curs.fetchone()
    if room_status and room_status[0].lower() == "free":
        status = "Booked"
        book_query = 'INSERT INTO Bookings (booking_id, customer_id, room_no, check_in_date, check_out_date, status) VALUES (%s, %s, %s, %s, %s, %s);'
        curs.execute(book_query, (f"B{room_no}", customer_id, room_no, check_in_date, check_out_date, status))
        cnx.commit()
        update_room_status_query = 'UPDATE Rooms SET status = %s WHERE room_no = %s;'
        curs.execute(update_room_status_query, (status, room_no))
        cnx.commit()
        messagebox.showinfo("Success", f"Room {room_no} has been booked for customer_id {customer_id} with meals {'included' if meal_preference.lower() == 'yes' else 'not included'}.")
    else:
        messagebox.showerror("Error", f"Room {room_no} is not available for booking.")

# Create the main window
root = Tk()
root.title("Hotel Management System")

# Add buttons for different functionalities
Button(root, text="Update Customer Details", command=update_customer_details).pack(pady=10)
Button(root, text="Update Employee Details", command=update_employee_details).pack(pady=10)
Button(root, text="Delete Customer Details", command=delete_customer_details).pack(pady=10)
Button(root, text="Delete Employee Details", command=delete_employee_details).pack(pady=10)
Button(root, text="Check Room Status", command=check_status).pack(pady=10)
Button(root, text="Book Room", command=book_room).pack(pady=10)
Button(root, text="Exit", command=root.quit).pack(pady=10)

root.mainloop()

# Close the database connection
cnx.close()
